In [30]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn



In [32]:
heartdis = pd.read_csv("heart_disease_uci.csv")
heartdis = heartdis.drop(columns=["id", "dataset", "ca"]) # Removed unimportant data. 
heartdis["num"] = (heartdis["num"] > 0).astype(int) # To increase prediction probabiliy 0 no heart disease and 1 has some level of heart disease 



In [34]:
string_data = heartdis.select_dtypes(include=["object"]).columns.tolist()
numerical_data = heartdis.select_dtypes(include=["float64", "int64"]).columns.tolist()
heartdis = pd.get_dummies(heartdis, columns=string_data) 



In [36]:
# Normalize numerical features since string_data is discrete data no need to normalize the data this is because it either has 100% prob or 0 % prob. 
mean = heartdis[numerical_data].mean()
std = heartdis[numerical_data].std()
heartdis[numerical_data] = (heartdis[numerical_data] - mean) / std



In [38]:
heartdis = heartdis.fillna(0) # need to replace the Nan data that still exsists in the numerical_data 



In [40]:
target_y = heartdis["num"].astype(np.float32).values
X = heartdis.drop(columns=["num"]).values.astype(np.float32)



In [42]:
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(target_y, dtype=torch.float32).unsqueeze(1)



In [44]:
torch.manual_seed(1234)

input_dim = X_tensor.shape[1]
hidden_neurons = 16
output_dim = 1 # Output is only one dimention becuase either 0 or 1 

# Initialize weights and biases for hidden and output layer
W1 = torch.randn(input_dim, hidden_neurons, dtype=torch.float32, requires_grad=True)
b1 = torch.zeros(hidden_neurons, dtype=torch.float32, requires_grad=True)

W2 = torch.randn(hidden_neurons, output_dim, dtype=torch.float32, requires_grad=True)
b2 = torch.zeros(output_dim, dtype=torch.float32, requires_grad=True)




In [46]:
class MLP:
    def __init__(self, input_dim, hidden_dim, output_dim):
        # Initialize weights and biases
        self.W1 = torch.randn(input_dim, hidden_dim, dtype=torch.float32, requires_grad=True)
        self.b1 = torch.zeros(hidden_dim, dtype=torch.float32, requires_grad=True)

        self.W2 = torch.randn(hidden_dim, output_dim, dtype=torch.float32, requires_grad=True)
        self.b2 = torch.zeros(output_dim, dtype=torch.float32, requires_grad=True)

    def tanh(self, x):
        return torch.tanh(x)

    def sigmoid(self, x):
        return 1 / (1 + torch.exp(-x))

    def binary_cross_entropy(self, y_pred, y_true):
        eps = 1e-7
        y_pred = torch.clamp(y_pred, eps, 1 - eps)
        return - (y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred)).mean()

    def forward(self, x):
        z1 = x @ self.W1 + self.b1
        a1 = self.tanh(z1)
        z2 = a1 @ self.W2 + self.b2
        y_pred = self.sigmoid(z2)
        return y_pred

    def parameters(self):
        return [self.W1, self.b1, self.W2, self.b2]

    def zero_grad(self):
        for param in self.parameters():
            if param.grad is not None:
                param.grad.zero_()


In [56]:
model = MLP(input_dim=X_tensor.shape[1], hidden_dim=16, output_dim=1)
learning_rate = 0.01
epochs = 1000

for epoch in range(epochs):
    # Forward pass
    y_pred = model.forward(X_tensor)
    loss = model.binary_cross_entropy(y_pred, Y_tensor)

    # Backward
    loss.backward()

    # Update
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

    model.zero_grad()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 1.2965
Epoch 100, Loss: 0.9436
Epoch 200, Loss: 0.7670
Epoch 300, Loss: 0.6646
Epoch 400, Loss: 0.5992
Epoch 500, Loss: 0.5552
Epoch 600, Loss: 0.5246
Epoch 700, Loss: 0.5030
Epoch 800, Loss: 0.4871
Epoch 900, Loss: 0.4753


In [60]:
with torch.no_grad():
    y_pred = model.forward(X_tensor)
    predictions = (y_pred > 0.5).float()  # Convert probabilities to 0 or 1
    correct = (predictions == Y_tensor).float().sum()
    accuracy = correct / Y_tensor.shape[0]
    print(f"Final Training Accuracy: {accuracy:.4f}")


Final Training Accuracy: 0.7913


In [52]:
with torch.no_grad():
    y_sample_pred = model.forward(X_tensor[0].unsqueeze(0))
    print(f"Predicted probability: {y_sample_pred.item():.4f}")


Predicted probability: 0.7042
